In [3]:
!pip install arxiv


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: C:\Users\hwnam\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [31]:
import urllib
import arxiv
import requests
import json
import csv
import pandas as pd
from collections import Counter, defaultdict
import numpy as np # for array manipulation
import matplotlib.pyplot as plt # for data visualization
%matplotlib inline 
import datetime
import json
import tarfile
import os
import re
import shutil
import feedparser
import urllib.parse

In [2]:
# read filtered_papers.json
with open('filtered_papers.json', 'r', encoding="utf-8") as f:
    papers = json.load(f)

print(f"Number of papers: {len(papers)}")

Number of papers: 2252


In [3]:
paper = papers[0]
query = paper["title"]
print(f"Query: {query}")

search = arxiv.Search(
    query=query,
    max_results=1,
    sort_by=arxiv.SortCriterion.Relevance,
)

for result in search.results():
    title = result.title
    paper_id = result.entry_id.split("/")[-1]
    print(f"Found paper: {title}")
    print(f"Paper_id: {paper_id}")

    if query == title:
        max_try = 0
        print("Paper found in arXiv")
        
        url = f"https://arxiv.org/e-print/{paper_id}"
        response = requests.get(url)
        while max_try < 5:
            if response.status_code == 200:
                with open(f"{paper_id}.tar.gz", "wb") as f:
                    f.write(response.content)
                print("Paper found source file")
                break
            else:
                print("Paper not found source file")
                max_try += 1
                response = requests.get(url)

Query: Residual Attention Network for Image Classification


C:\Users\남현원\AppData\Local\Temp\ipykernel_4200\3935089147.py:11: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Found paper: Residual Attention Network for Image Classification
Paper_id: 1704.06904v1
Paper found in arXiv
Paper found source file


In [4]:
tar_path = "1704.06904v1.tar.gz"

#with tarfile.open(tar_path, "r:gz") as tar:
    #tar.extractall("Example")


for fname in os.listdir("Example"):
    if fname.endswith(".tex"):
        print(fname)
        tex_file = fname

attention-net_camera_ready_merge_final.tex


In [15]:
def split_section(tex_file):
    sections = []
    match = re.search(r"\\begin{abstract}(?:\[[^\]]*\])?(.*?)\\end{abstract}", tex_file, re.DOTALL)
    section_name = "abstract"
    if match:
        content = match.group(1).strip()
        sections.append((section_name, content))
    else:
        print("No abstract found")

    pattern = r"\\section\*?{([^}]+)}"
    matches = list(re.finditer(pattern, tex_file))
    print(f"Number of sections: {len(matches)}")
    
    for i, match in enumerate(matches):
        section_name = match.group(1).strip()
        
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(tex_file)
        
        content = tex_file[start:end].strip()
        sections.append((section_name, content))
        
    return sections

In [16]:
with open("Example/" + tex_file, "r", encoding="utf-8") as f:
    latex_text = f.read()

splitBy_section = split_section(latex_text)

splitBy_section

Number of sections: 5


[('abstract',
  '%Mixed nature of human attention has been proposed in the literature of biology and been applied to sequential learning task using RNN and LSTM.\nIn this work, we propose ``Residual Attention Network", a convolutional neural network using attention mechanism which can incorporate with state-of-art feed forward network architecture in an end-to-end training fashion.\n%\nOur Residual Attention Network is built by stacking Attention Modules which generate attention-aware features. The attention-aware features from different modules change adaptively as layers going deeper. Inside each Attention Module, bottom-up top-down feedforward structure is used to unfold the feedforward and feedback attention process into a single feedforward process. Importantly, we propose attention residual learning to train very deep Residual Attention Networks which can be easily scaled up to hundreds of layers.\n\nExtensive analyses are conducted on CIFAR-10 and CIFAR-100 datasets to verify th

In [17]:
json_data = []
for section_name, content in splitBy_section:
    cleaned_content = re.sub(r"\\begin{([a-zA-Z*]+)}(?:\[[^\]]*\])?.*?\\end{\1}"
, "", content, flags=re.DOTALL)
    cleaned_content = cleaned_content.strip()
    
    lines = cleaned_content.splitlines()
    cleaned_content = [line for line in lines if line.strip() and not line.startswith("%")]
    cleaned_content = "\n".join(cleaned_content)
    
    json_data.append({
        "section_name": section_name,
        "content": cleaned_content
    })
    
with open("Example/cleaned_data.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

In [19]:
print(json_data)

[{'section_name': 'abstract', 'content': 'In this work, we propose ``Residual Attention Network", a convolutional neural network using attention mechanism which can incorporate with state-of-art feed forward network architecture in an end-to-end training fashion.\nOur Residual Attention Network is built by stacking Attention Modules which generate attention-aware features. The attention-aware features from different modules change adaptively as layers going deeper. Inside each Attention Module, bottom-up top-down feedforward structure is used to unfold the feedforward and feedback attention process into a single feedforward process. Importantly, we propose attention residual learning to train very deep Residual Attention Networks which can be easily scaled up to hundreds of layers.\nExtensive analyses are conducted on CIFAR-10 and CIFAR-100 datasets to verify the effectiveness of every module mentioned above. Our Residual Attention Network achieves state-of-the-art object recognition p

# Make all data

In [29]:
json_data = []

for paper in papers:
    title = paper["title"]

    search = arxiv.Search(
        query=title,
        max_results=1,
        sort_by=arxiv.SortCriterion.Relevance,
    )

    for result in search.results():
        title = result.title
        paper_id = result.entry_id.split("/")[-1]
        print(f"Found paper: {title}")
        print(f"Paper_id: {paper_id}")

        if query == title:
            max_try = 0
            print("Paper found in arXiv")

            url = f"https://arxiv.org/e-print/{paper_id}"
            response = requests.get(url)
            while max_try < 5:
                if response.status_code == 200:
                    with open(f"{paper_id}.tar.gz", "wb") as f:
                        f.write(response.content)
                    print("Paper found source file")
                    break
                else:
                    print("Paper not found source file")
                    max_try += 1
                    response = requests.get(url)

            tar_path = f"{paper_id}.tar.gz"
            with tarfile.open(tar_path, "r:gz") as tar:
                tar.extractall("SourceFiles")
            os.remove(tar_path)

            for fname in os.listdir("SourceFiles"):
                if fname.endswith(".tex"):
                    print(fname)
                    tex_file = fname

            with open("SourceFiles/" + tex_file, "r", encoding="utf-8") as f:
                latex_text = f.read()
            
            splitBy_section = split_section(latex_text)
            
            json_data.append({
                    "title": title,
                    "paper_id": paper_id,
                    "sections": []
            })
            for section_name, content in splitBy_section:
                cleaned_content = re.sub(r"\\begin{([a-zA-Z*]+)}(?:\[[^\]]*\])?.*?\\end{\1}", "", content, flags=re.DOTALL)
                cleaned_content = cleaned_content.strip()

                lines = cleaned_content.splitlines()
                cleaned_content = [line for line in lines if line.strip() and not line.startswith("%")]
                cleaned_content = "\n".join(cleaned_content)

                json_data[-1]["sections"].append({
                    section_name : content
                })

            shutil.rmtree("SourceFiles")
        
        else:
            print("Paper not found in arXiv")
            print(f"query: {query}")
            print(f"title: {title}")
            continue
    print("\n")


C:\Users\남현원\AppData\Local\Temp\ipykernel_4200\2420375828.py:12: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


Found paper: Residual Attention Network for Image Classification
Paper_id: 1704.06904v1
Paper found in arXiv
Paper found source file
attention-net_camera_ready_merge_final.tex
Number of sections: 5


Found paper: Privacy-Preserving Image Classification Using Vision Transformer
Paper_id: 2205.12041v1
Paper not found in arXiv
query: Residual Attention Network for Image Classification
title: Privacy-Preserving Image Classification Using Vision Transformer


Found paper: Classification of optics-free images with deep neural networks
Paper_id: 2011.05132v1
Paper not found in arXiv
query: Residual Attention Network for Image Classification
title: Classification of optics-free images with deep neural networks




ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [32]:
json_data = []

for paper in papers:
    query = paper["title"].strip()
    encoded_query = urllib.parse.quote(query)

    # 정확히 제목 기반으로 arXiv 검색
    url = f"http://export.arxiv.org/api/query?search_query=ti:{encoded_query}&max_results=5"
    feed = feedparser.parse(url)

    found_match = False

    for entry in feed.entries:
        arxiv_title = entry.title.strip()

        if query.lower() == arxiv_title.lower():
            found_match = True
            paper_id = entry.id.split("/")[-1]
            print(f"🎯 Found paper: {arxiv_title}")
            print(f"Paper_id: {paper_id}")

            # 논문 소스 다운로드
            url = f"https://arxiv.org/e-print/{paper_id}"
            max_try = 0
            while max_try < 5:
                response = requests.get(url)
                if response.status_code == 200:
                    with open(f"{paper_id}.tar.gz", "wb") as f:
                        f.write(response.content)
                    print("📦 Source file downloaded")
                    break
                else:
                    print("Retrying download...")
                    max_try += 1
            else:
                print("❌ Failed to download after 5 tries")
                continue

            # 압축 해제
            tar_path = f"{paper_id}.tar.gz"
            try:
                with tarfile.open(tar_path, "r:gz") as tar:
                    tar.extractall("SourceFiles")
                os.remove(tar_path)
            except tarfile.ReadError:
                print(f"❌ {tar_path} is not a valid gzip file. Skipping...")
                continue

            # .tex 파일 추출
            tex_file = None
            for fname in os.listdir("SourceFiles"):
                if fname.endswith(".tex"):
                    tex_file = fname
                    break

            if not tex_file:
                print("❌ No .tex file found.")
                shutil.rmtree("SourceFiles")
                continue

            with open(os.path.join("SourceFiles", tex_file), "r", encoding="utf-8") as f:
                latex_text = f.read()

            splitBy_section = split_section(latex_text)

            paper_entry = {
                "title": arxiv_title,
                "paper_id": paper_id,
                "sections": []
            }

            for section_name, content in splitBy_section:
                cleaned_content = re.sub(r"\\begin{([a-zA-Z*]+)}(?:\[[^\]]*\])?.*?\\end{\1}", "", content, flags=re.DOTALL)
                lines = cleaned_content.strip().splitlines()
                cleaned_content = "\n".join([line for line in lines if line.strip() and not line.strip().startswith("%")])
                paper_entry["sections"].append({section_name: cleaned_content})

            json_data.append(paper_entry)
            shutil.rmtree("SourceFiles")
            break  # 일치하는 제목 찾으면 종료

    if not found_match:
        print("⚠ 제목과 정확히 일치하는 논문을 찾을 수 없습니다.")
        print(f"query: {query}")
    print("\n")

🎯 Found paper: Residual Attention Network for Image Classification
Paper_id: 1704.06904v1
📦 Source file downloaded
Number of sections: 5


⚠ 제목과 정확히 일치하는 논문을 찾을 수 없습니다.
query: CrossViT: Cross-Attention Multi-Scale Vision Transformer for Image Classification


⚠ 제목과 정확히 일치하는 논문을 찾을 수 없습니다.
query: Multi-column deep neural networks for image classification


⚠ 제목과 정확히 일치하는 논문을 찾을 수 없습니다.
query: Locality-constrained Linear Coding for image classification


⚠ 제목과 정확히 일치하는 논문을 찾을 수 없습니다.
query: Rethinking Few-Shot Image Classification: a Good Embedding Is All You Need?


⚠ 제목과 정확히 일치하는 논문을 찾을 수 없습니다.
query: Bag of Tricks for Image Classification with Convolutional Neural Networks


🎯 Found paper: Big Self-Supervised Models Advance Medical Image Classification
Paper_id: 2101.05224v2
📦 Source file downloaded
Number of sections: 7


🎯 Found paper: Understanding Robustness of Transformers for Image Classification
Paper_id: 2103.14586v2
📦 Source file downloaded
Number of sections: 0


⚠ 제목과 정확히 일

In [26]:
json_data

[{'title': 'Residual Attention Network for Image Classification',
  'paper_id': '1704.06904v1',
  'sections': [{'abstract': '%Mixed nature of human attention has been proposed in the literature of biology and been applied to sequential learning task using RNN and LSTM.\nIn this work, we propose ``Residual Attention Network", a convolutional neural network using attention mechanism which can incorporate with state-of-art feed forward network architecture in an end-to-end training fashion.\n%\nOur Residual Attention Network is built by stacking Attention Modules which generate attention-aware features. The attention-aware features from different modules change adaptively as layers going deeper. Inside each Attention Module, bottom-up top-down feedforward structure is used to unfold the feedforward and feedback attention process into a single feedforward process. Importantly, we propose attention residual learning to train very deep Residual Attention Networks which can be easily scaled u

In [33]:
with open("cleaned_data.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)